# Gossiping con protocolo push donde se converge al valor máximo

El envio de mensajes es solo a un vecino aleatorio

**Recuerda cambiar el nombre de los agentes**

In [6]:
import datetime
import json
import random
import time
import asyncio
import spade

class PushPullAgent(spade.agent.Agent):
    def __init__(self, jid, password, k, type=0):
        super().__init__(jid, password)
        self.k = k
        self.message_count = 0
        self.type = type

    async def setup(self):
        self.value = random.randint(1, 1000)
        start_at = datetime.datetime.now() + datetime.timedelta(seconds=5)
        
        # Comportamiento de Push: enviar valores a otros agentes
        self.add_behaviour(self.PushBehaviour(period=2, start_at=start_at))
        
        # Comportamiento de Pull: solicitar valores a otros agentes
        self.add_behaviour(self.PullBehaviour(period=2, start_at=start_at))

        # Template para gestionar solicitudes y respuestas
        template_request = spade.template.Template(metadata={"performative": "REQUEST"})
        template_response = spade.template.Template(metadata={"performative": "RESPONSE"})
        self.add_behaviour(self.RequestHandler(), template_request)
        self.add_behaviour(self.ResponseHandler(), template_response)

    def add_value(self, received_value):
        # Actualiza el valor tomando la media entre el valor propio y el recibido
        if self.type == 0:
            self.value = (self.value + received_value) / 2
        else:
            self.value = max(self.value, received_value)

    def add_contacts(self, contact_list):
        self.contacts = [c.jid for c in contact_list if c.jid != self.jid]
        self.length = len(self.contacts)
        if self.k > self.length:
            self.k = self.length

    # Comportamiento de Push: enviar valores a otros agentes
    class PushBehaviour(spade.behaviour.PeriodicBehaviour):
        async def run(self):
            k = self.agent.k
            random_contacts = random.sample(self.agent.contacts, k)
            
            # Enviar valor a contactos seleccionados
            for jid in random_contacts:
                body = json.dumps({"value": self.agent.value, "timestamp": time.time()})
                msg = spade.message.Message(to=str(jid), body=body, metadata={"performative": "PUSH"})
                await self.send(msg)
                self.agent.message_count += 1

    # Comportamiento de Pull: solicitar valores de otros agentes
    class PullBehaviour(spade.behaviour.PeriodicBehaviour):
        async def run(self):
            k = self.agent.k
            random_contacts = random.sample(self.agent.contacts, k)
            
            # Solicitar valores de contactos seleccionados
            for jid in random_contacts:
                msg = spade.message.Message(to=str(jid), metadata={"performative": "REQUEST"})
                await self.send(msg)
                self.agent.message_count += 1

    # Manejar solicitudes (REQUEST) y enviar respuestas (RESPONSE)
    class RequestHandler(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=2)
            if msg:
                # Gestiona una solicitud y responde con el valor actual
                response = spade.message.Message(
                    to=str(msg.sender),
                    body=json.dumps({"value": self.agent.value}),
                    metadata={"performative": "RESPONSE"}
                )
                await self.send(response)
                self.agent.message_count += 1

    # Manejar respuestas (RESPONSE) y actualizar valor
    class ResponseHandler(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=2)
            if msg:
                # Gestiona una respuesta y actualiza el valor del agente
                body = json.loads(msg.body)
                self.agent.add_value(body["value"])



In [14]:
# Parámetros del experimento
count = 50
k = 5
agents = []
print(f"Creando {count} agentes con k={k}")

# Crear agentes Push-Pull
for x in range(1, count + 1):
    agents.append(PushPullAgent(f"push_pull_agent_{x}@localhost", "test", k, type=1))

# Tiempo para permitir el registro
await asyncio.sleep(3)

# Añadir contactos a cada agente
for ag in agents:
    ag.add_contacts(agents)

# Lanzar agentes
tasks = [ag.start() for ag in agents]
await asyncio.gather(*tasks)

# Medir tiempo y mensajes
start_time = time.time()
while True:
    await asyncio.sleep(1)
    status = [ag.value for ag in agents]
    print(f"STATUS: {status}")
    if len(set(status)) <= 1:
        break
end_time = time.time()

# Calcular métricas
total_time = end_time - start_time
total_messages = sum(ag.message_count for ag in agents)

# Mostrar resultados
print(f"Gossip terminado en {total_time:.2f} seconds.")
print(f"Mensajes enviados {total_messages}.")

# Parar agentes
for ag in agents:
    await ag.stop()
print("Agentes parados")

Creando 50 agentes con k=5
STATUS: [894, 918, 836, 918, 903, 989, 894, 699, 691, 989, 903, 918, 798, 918, 804, 911, 911, 804, 989, 903, 989, 918, 911, 903, 903, 911, 989, 918, 836, 903, 894, 911, 918, 903, 918, 903, 836, 989, 989, 911, 989, 536, 836, 918, 918, 918, 911, 989, 918, 680]
STATUS: [894, 918, 836, 918, 903, 989, 894, 699, 691, 989, 903, 918, 798, 918, 804, 911, 911, 804, 989, 903, 989, 918, 911, 903, 903, 911, 989, 918, 836, 903, 894, 911, 918, 903, 918, 903, 836, 989, 989, 911, 989, 536, 836, 918, 918, 918, 911, 989, 918, 680]
STATUS: [989, 989, 989, 918, 989, 989, 918, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 918, 918, 989, 989, 989, 989, 918, 989, 989, 989, 989, 989, 918, 989, 989, 989, 989, 989, 989, 989, 989]
STATUS: [989, 989, 989, 918, 989, 989, 918, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 989, 918, 918, 989, 989, 989, 989, 918, 989, 98

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
Future exception was never retrieved
future: <Future finished exception=IqTimeout(<iq id="f546000cd969434eb26bea0a9b700f6c" type="get"><query xmlns="jabber:iq:roster" /></iq>)>
slixmpp.exceptions.IqTimeout: <iq id="f546000cd969434eb26bea0a9b700f6c" type="get"><query xmlns="jabber:iq:roster" /></iq>
Future exception was never retrieved
future: <Future finished exception=IqTimeout(<iq id="99

In [2]:
# Parámetros del experimento
count = 50
k = 5
agents = []
print(f"Creando {count} agentes con k={k}")

# Crear agentes Push-Pull
for x in range(1, count + 1):
    agents.append(PushPullAgent(f"push_pull_agent_{x}@localhost", "test", k, type=0))

# Tiempo para permitir el registro
await asyncio.sleep(3)

# Añadir contactos a cada agente
for ag in agents:
    ag.add_contacts(agents)

# Lanzar agentes
tasks = [ag.start() for ag in agents]
await asyncio.gather(*tasks)

# Medir tiempo y mensajes
start_time = time.time()
while True:
    await asyncio.sleep(1)
    status = [ag.value for ag in agents]
    print(f"STATUS: {status}")
    if len(set(status)) <= 1:
        break
end_time = time.time()

# Calcular métricas
total_time = end_time - start_time
total_messages = sum(ag.message_count for ag in agents)

# Mostrar resultados
print(f"Gossip terminado en {total_time:.2f} seconds.")
print(f"Mensajes enviados {total_messages}.")

# Parar agentes
for ag in agents:
    await ag.stop()
print("Agentes parados")

Creando 50 agentes con k=5
STATUS: [312, 364, 960, 886, 974, 680, 566, 217, 265, 10, 979, 309, 622, 714, 482, 997, 908, 924, 955, 855, 995, 236, 499, 840, 259, 863, 484, 220, 863, 130, 807, 869, 806, 38, 78, 220, 173, 324, 258, 840, 837, 590, 162, 969, 394, 71, 33, 982, 524, 404]
STATUS: [312, 364, 960, 886, 974, 680, 566, 217, 265, 10, 979, 309, 622, 714, 482, 997, 908, 924, 955, 855, 995, 236, 499, 840, 259, 863, 484, 220, 863, 130, 807, 869, 806, 38, 78, 220, 173, 324, 258, 840, 837, 590, 162, 969, 394, 71, 33, 982, 524, 404]
STATUS: [312, 364, 960, 886, 974, 680, 566, 217, 265, 10, 979, 309, 622, 714, 482, 997, 908, 924, 955, 855, 995, 236, 499, 840, 259, 863, 484, 220, 863, 130, 807, 869, 806, 38, 78, 220, 173, 324, 258, 840, 837, 590, 162, 969, 394, 71, 33, 982, 524, 404]
STATUS: [312, 364, 960, 886, 974, 680, 566, 217, 265, 10, 979, 309, 622, 714, 482, 997, 908, 924, 955, 855, 995, 236, 499, 840, 259, 863, 484, 220, 863, 130, 807, 869, 806, 38, 78, 220, 173, 324, 258, 840, 837, 

CancelledError: 

In [3]:
async def run_agents(t,k=2,a=50, path="result_push_pull.txt"):
    agents = []
    print(f"Creando {a} agentes con k={k} en type {t}")

    # Crear agentes Push-Pull
    for x in range(1, a + 1):
        agents.append(PushPullAgent(f"pull_agent_{x}@localhost", "test", k, type=t))

    # Tiempo para permitir el registro
    await asyncio.sleep(3)

    # Añadir contactos a cada agente
    for ag in agents:
        ag.add_contacts(agents)

    # Lanzar agentes
    tasks = [ag.start() for ag in agents]
    await asyncio.gather(*tasks)

    # Medir tiempo y mensajes
    start_time = time.time()
    while True:
        await asyncio.sleep(1)
        status = [ag.value for ag in agents]
        if len(set(status)) <= 1:
            break
    end_time = time.time()

    # Calcular métricas
    total_time = end_time - start_time
    total_messages = sum(ag.message_count for ag in agents)

    with open(path,"a") as f:
        f.write(f"{t},{k},{a},{total_messages},{total_time}\n")

    # Mostrar resultados
    print(f"Gossip terminado en {total_time:.2f} seconds.")
    print(f"Mensajes enviados {total_messages}.")

    # Parar agentes
    for ag in agents:
        await ag.stop()
    print("Agentes parados")


In [12]:
for t in [1]:
     for k in [1,3,5,10]:
         await run_agents(t, k=k)
         await asyncio.sleep(1)
        
    #for ag in [10,20,50,100,200]:
    #    await run_agents(t=t,a=ag)
    #    await asyncio.sleep(2)


Creando 50 agentes con k=1 en type 1


Gossip terminado en 21.01 seconds.
Mensajes enviados 1350.
Agentes parados
Creando 50 agentes con k=3 en type 1
Gossip terminado en 13.03 seconds.
Mensajes enviados 2228.
Agentes parados
Creando 50 agentes con k=5 en type 1
Gossip terminado en 10.03 seconds.
Mensajes enviados 2250.
Agentes parados
Creando 50 agentes con k=10 en type 1


No behaviour matched for message: <message to="pull_agent_2@localhost" from="pull_agent_9@localhost" thread="None" metadata={'performative': 'PUSH'}>
{"value": 978, "timestamp": 1736356091.6241221}
</message>
No behaviour matched for message: <message to="pull_agent_3@localhost" from="pull_agent_6@localhost" thread="None" metadata={'performative': 'PUSH'}>
{"value": 978, "timestamp": 1736356091.642802}
</message>
No behaviour matched for message: <message to="pull_agent_3@localhost" from="pull_agent_23@localhost" thread="None" metadata={'performative': 'PUSH'}>
{"value": 978, "timestamp": 1736356091.6451929}
</message>
No behaviour matched for message: <message to="pull_agent_3@localhost" from="pull_agent_18@localhost" thread="None" metadata={'performative': 'PUSH'}>
{"value": 978, "timestamp": 1736356091.646347}
</message>
No behaviour matched for message: <message to="pull_agent_2@localhost" from="pull_agent_18@localhost" thread="None" metadata={'performative': 'PUSH'}>
{"value": 978

Gossip terminado en 8.93 seconds.
Mensajes enviados 3000.
Agentes parados


In [8]:
for t in [0]:
    for k in [1,3,5,10]:
        await run_agents(t, k=k)
        await asyncio.sleep(2)

Creando 50 agentes con k=1 en type 0
Gossip terminado en 195.53 seconds.
Mensajes enviados 14400.
Agentes parados
Creando 50 agentes con k=3 en type 0
Gossip terminado en 103.35 seconds.
Mensajes enviados 22500.
Agentes parados
Creando 50 agentes con k=5 en type 0
Gossip terminado en 79.46 seconds.
Mensajes enviados 28500.
Agentes parados
Creando 50 agentes con k=10 en type 0
Gossip terminado en 73.51 seconds.
Mensajes enviados 52500.
Agentes parados


In [9]:
for t in [0]:
    for ag in [10,20,50]:
        await run_agents(t=t,a=ag)
        await asyncio.sleep(1)

Creando 10 agentes con k=2 en type 0
Gossip terminado en 91.15 seconds.
Mensajes enviados 2640.
Agentes parados
Creando 20 agentes con k=2 en type 0
Gossip terminado en 107.13 seconds.
Mensajes enviados 6240.
Agentes parados
Creando 50 agentes con k=2 en type 0
Gossip terminado en 111.52 seconds.
Mensajes enviados 16200.
Agentes parados


In [11]:
await run_agents(0,a=100)

Creando 100 agentes con k=2 en type 0
Gossip terminado en 121.80 seconds.
Mensajes enviados 35400.
Agentes parados


In [ ]:
await run_agents(0,a=200)